In [1]:
import geopandas as gpd 
import numpy as np

In [36]:
def prep_poly_geom(path_open, path_save, hull=False): 
    '''
    Opens a shapefile with a polygon boundary 
    and saves wkt_geom to text file 

    IMPORTANT: the feature class must only have one feature (shape)! 

    args:
        path_open - str - dir with a shapefile 
        path_save - str - dir to save the txt to 
    '''

    gdf = gpd.read_file(path_open) 
    print('opened file with shape:', gdf.shape)

    # check if there is only one feazture in the data
    assert gdf.shape[0] == 1, "Feature class contains more than ONE geometry" 

    # reproject to pseudo merc 
    #gdf = gdf.to_crs('epsg:3857')

    if hull: 
        geom = gdf.geometry.convex_hull.to_wkt()
    else: 
        # simplify to 100 meters
        geom = gdf.geometry.simplify(0.000009).to_wkt()
    
    # print(type(geom))

    # reproject back to GCS
    # gdf = gdf.to_crs('epsg:4326')

    text = geom[0]
    #print(text[0])
    #print(type(text))
    
    with open(path_save, 'w') as f: 
        f.write(text)
        f.close()


if __name__=='__main__': 
    path_open = 'C:/Users/barguzin/Documents/Github/rwmp_pipelines/rwmp_study_area.geojson'
    path_save = 'C:/Users/barguzin/Documents/Github/rwmp_pipelines/rwmp_study_area.txt'
    prep_poly_geom(path_open, path_save, hull=True)

opened file with shape: (1, 14)


In [117]:
gdf = gpd.read_file('C:/Users/barguzin/Documents/Github/rwmp_pipelines/roadsevac_demo2_epsg26910.geojson')

In [118]:
x  = gdf.geometry.to_wkt()
type(x[0])

str

In [2]:
from OSMPythonTools.overpass import Overpass
overpass = Overpass()
result = overpass.query('way["name"="Stephansdom"]; out body;')

[overpass] downloading data: [timeout:25][out:json];way["name"="Stephansdom"]; out body;


In [6]:
stephansdom = result.elements()[0]

#stephansdom.tag('name:en')

# "Saint Stephen's Cathedral"
'%s %s, %s %s' % (stephansdom.tag('addr:street'), stephansdom.tag('addr:housenumber'), stephansdom.tag('addr:postcode'), stephansdom.tag('addr:city'))

# 'Stephansplatz 3, 1010 Wien'
#stephansdom.tag('building')
# 'cathedral'
#stephansdom.tag('denomination')
# 'catholic'

'Stephansplatz 3, 1010 Wien'

In [114]:
coords = re.findall(r'-?\d+\.\d+', x[0])

lng, lat = [],[]
reversed = []
flat = []

for i in coords:
    #print(type(float(i)))
    if float(i)<0:
        lng.append(i)
    else: 
        lat.append(i)

for i,j in zip(lng,lat): 
    reversed.append([j,i])

flat = [item for sublist in reversed for item in sublist]

s = ' '.join(flat) 

In [7]:
def wkt_to_api(wkt):
    '''
    Converts a string with wkt geometry to polygon coordinates 
    suitable for calls to Overpass API 

    Args: 
        wkt - str - wkt geometry from geopandas / pygeos 
    '''

    coords = re.findall(r'-?\d+\.\d+', x[0])

    lng, lat = [],[]
    reversed = []
    flat = []

    for i in coords:
        #print(type(float(i)))
        if float(i)<0:
            lng.append(i)
        else: 
            lat.append(i)

    for i,j in zip(lng,lat): 
        reversed.append([j,i])

    flat = [item for sublist in reversed for item in sublist]

    s = ' '.join(flat) 

    return s

    

In [5]:
import overpass

api = overpass.API(timeout=600)
response = api.get('node["name"="Salt Lake City"]')

In [15]:
import requests
import json

overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
/*
Download OSM data within a poly 
*/

[out:json][timeout:100];

// gather results
(
  node(poly: "34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");
  way(poly: "34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");
  relation(poly: "34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");
);
// print results
out body;
>;
out skel qt;
"""

# get data
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [20]:
with open('data.xml', 'w') as f: 
    f.write(response.text)

In [21]:
import requests
import json

overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json][timeout:100];
(
  node(poly: "34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");
  way(poly: "34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");
  relation(poly: "34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");
);
out body;
>;
out skel qt;
"""

# get data
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response.url)

http://overpass-api.de/api/interpreter?data=%0A%5Bout%3Ajson%5D%5Btimeout%3A100%5D%3B%0A%28%0A++node%28poly%3A+%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.44592492359789+-119.70680828448938+34.446500639738005+-119.72738885839088+34.44858311972544+-119.73101458783302%22%29%3B%0A++way%28poly%3A+%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.44592492359789+-119.70680828448938+34.446500639738005+-119.72738885839088+34.44858311972544+-119.73101458783302%22%29%3B%0A++relation%28poly%3A+%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.445924923597

In [29]:
# concat the query url 

interpret = "http://overpass-api.de/api/interpreter?data="
output_format = "[out:xml][timeout:100];"
nodes = '(node(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");'
ways = 'way(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");'
relations = 'relation(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");)'
res = ";out;>;out skel qt;"

url = interpret + output_format + nodes + ways + relations + res

print(url)


http://overpass-api.de/api/interpreter?data=[out:xml][timeout:100];(node(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");way(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");relation(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.448583

In [33]:
# resp = requests.get(url) 

# with open('mission.xml', 'w') as f:
#     f.write(resp.text)

In [40]:
# import urllib.request
# from urllib.parse import quote

# urllib.request.urlretrieve(quote(url), "my_file")

In [41]:
import re 

escaped = re.escape(url) 
print(escaped)

http://overpass\-api\.de/api/interpreter\?data=\[out:xml\]\[timeout:100\];\(node\(poly:"34\.44858311972544\ \-119\.73101458783302\ 34\.45239888013674\ \-119\.72767225216123\ 34\.456219261515486\ \-119\.71929761637043\ 34\.46525906534552\ \-119\.71437116474895\ 34\.465304752065684\ \-119\.70402501878468\ 34\.44592492359789\ \-119\.70680828448938\ 34\.446500639738005\ \-119\.72738885839088\ 34\.44858311972544\ \-119\.73101458783302"\);way\(poly:"34\.44858311972544\ \-119\.73101458783302\ 34\.45239888013674\ \-119\.72767225216123\ 34\.456219261515486\ \-119\.71929761637043\ 34\.46525906534552\ \-119\.71437116474895\ 34\.465304752065684\ \-119\.70402501878468\ 34\.44592492359789\ \-119\.70680828448938\ 34\.446500639738005\ \-119\.72738885839088\ 34\.44858311972544\ \-119\.73101458783302"\);relation\(poly:"34\.44858311972544\ \-119\.73101458783302\ 34\.45239888013674\ \-119\.72767225216123\ 34\.456219261515486\ \-119\.71929761637043\ 34\.46525906534552\ \-119\.71437116474895\ 34\.4653047520

In [43]:
from urllib.parse import quote

quote(url, safe='')

'http%3A%2F%2Foverpass-api.de%2Fapi%2Finterpreter%3Fdata%3D%5Bout%3Axml%5D%5Btimeout%3A100%5D%3B%28node%28poly%3A%2234.44858311972544%20-119.73101458783302%2034.45239888013674%20-119.72767225216123%2034.456219261515486%20-119.71929761637043%2034.46525906534552%20-119.71437116474895%2034.465304752065684%20-119.70402501878468%2034.44592492359789%20-119.70680828448938%2034.446500639738005%20-119.72738885839088%2034.44858311972544%20-119.73101458783302%22%29%3Bway%28poly%3A%2234.44858311972544%20-119.73101458783302%2034.45239888013674%20-119.72767225216123%2034.456219261515486%20-119.71929761637043%2034.46525906534552%20-119.71437116474895%2034.465304752065684%20-119.70402501878468%2034.44592492359789%20-119.70680828448938%2034.446500639738005%20-119.72738885839088%2034.44858311972544%20-119.73101458783302%22%29%3Brelation%28poly%3A%2234.44858311972544%20-119.73101458783302%2034.45239888013674%20-119.72767225216123%2034.456219261515486%20-119.71929761637043%2034.46525906534552%20-119.71437

In [46]:
# concat the query url 
# updated version 
interpret = "http://overpass-api.de/api/interpreter"
output_format = "[out:xml][timeout:100];"
nodes = '(node(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");'
ways = 'way(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");'
relations = 'relation(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");)'
res = ";out;>;out skel qt;"

url2 = output_format + nodes + ways + relations + res

print(url2)


[out:xml][timeout:100];(node(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");way(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302");relation(poly:"34.44858311972544 -119.73101458783302 34.45239888013674 -119.72767225216123 34.456219261515486 -119.71929761637043 34.46525906534552 -119.71437116474895 34.465304752065684 -119.70402501878468 34.44592492359789 -119.70680828448938 34.446500639738005 -119.72738885839088 34.44858311972544 -119.73101458783302"););out;>;out s

In [48]:
quote(url2)

'%5Bout%3Axml%5D%5Btimeout%3A100%5D%3B%28node%28poly%3A%2234.44858311972544%20-119.73101458783302%2034.45239888013674%20-119.72767225216123%2034.456219261515486%20-119.71929761637043%2034.46525906534552%20-119.71437116474895%2034.465304752065684%20-119.70402501878468%2034.44592492359789%20-119.70680828448938%2034.446500639738005%20-119.72738885839088%2034.44858311972544%20-119.73101458783302%22%29%3Bway%28poly%3A%2234.44858311972544%20-119.73101458783302%2034.45239888013674%20-119.72767225216123%2034.456219261515486%20-119.71929761637043%2034.46525906534552%20-119.71437116474895%2034.465304752065684%20-119.70402501878468%2034.44592492359789%20-119.70680828448938%2034.446500639738005%20-119.72738885839088%2034.44858311972544%20-119.73101458783302%22%29%3Brelation%28poly%3A%2234.44858311972544%20-119.73101458783302%2034.45239888013674%20-119.72767225216123%2034.456219261515486%20-119.71929761637043%2034.46525906534552%20-119.71437116474895%2034.465304752065684%20-119.70402501878468%2034.

In [49]:
import urllib.parse 
safe = urllib.parse.quote_plus(url2)
safe

'%5Bout%3Axml%5D%5Btimeout%3A100%5D%3B%28node%28poly%3A%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.44592492359789+-119.70680828448938+34.446500639738005+-119.72738885839088+34.44858311972544+-119.73101458783302%22%29%3Bway%28poly%3A%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.44592492359789+-119.70680828448938+34.446500639738005+-119.72738885839088+34.44858311972544+-119.73101458783302%22%29%3Brelation%28poly%3A%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.44592492359789+-119.70680828448938+34.446500639738005+-119.72738885839088+34.448

In [50]:
# this should work : 

"http://overpass-api.de/api/interpreter?data=" + safe 

# %5Bout%3Axml%5D%5Btimeout%3A100%5D%3B%28node%28poly%3A%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.44592492359789+-119.70680828448938+34.446500639738005+-119.72738885839088+34.44858311972544+-119.73101458783302%22%29%3Bway%28poly%3A%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.44592492359789+-119.70680828448938+34.446500639738005+-119.72738885839088+34.44858311972544+-119.73101458783302%22%29%3Brelation%28poly%3A%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.44592492359789+-119.70680828448938+34.446500639738005+-119.72738885839088+34.44858311972544+-119.73101458783302%22%29%3B%29%3Bout%3B%3E%3Bout+skel+qt%3B"

'http://overpass-api.de/api/interpreter?data=%5Bout%3Axml%5D%5Btimeout%3A100%5D%3B%28node%28poly%3A%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.44592492359789+-119.70680828448938+34.446500639738005+-119.72738885839088+34.44858311972544+-119.73101458783302%22%29%3Bway%28poly%3A%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.44592492359789+-119.70680828448938+34.446500639738005+-119.72738885839088+34.44858311972544+-119.73101458783302%22%29%3Brelation%28poly%3A%2234.44858311972544+-119.73101458783302+34.45239888013674+-119.72767225216123+34.456219261515486+-119.71929761637043+34.46525906534552+-119.71437116474895+34.465304752065684+-119.70402501878468+34.44592492359789+-119.70680828448938+3

In [ ]:
# to-do 
# ------------
# generate poly bounds from txt with coordinates 
# wrap query building into Python function